In [1]:
%cd 04_ExerciseCodeAndInstructions/intro_to_TF
%set_env CUDA_VISIBLE_DEVICES=1
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

print(tf.__version__)
print(keras.__version__)
print(tf.config.list_physical_devices("GPU"))


/project/nanocourse/SWE_OOP/train07/Module_2_VMullapudi/04_ExerciseCodeAndInstructions/intro_to_TF
env: CUDA_VISIBLE_DEVICES=1


2025-05-27 12:52:47.676999: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748368367.700370   40761 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748368367.707796   40761 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748368367.725888   40761 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748368367.725906   40761 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748368367.725908   40761 computation_placer.cc:177] computation placer alr

2.19.0
3.9.2
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0


In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(32, 3, padding="valid", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10),
    ]
)


I0000 00:00:1748368374.380246   40761 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15506 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:82:00.0, compute capability: 6.0


In [4]:
print(model.summary())


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       131,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

None


In [5]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)


In [6]:
model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)


Epoch 1/10


I0000 00:00:1748368377.708200   41031 service.cc:152] XLA service 0x2aaeac00a0f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748368377.708236   41031 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-05-27 12:52:57.750355: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748368377.972682   41031 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-05-27 12:52:58.281170: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.9 = (f32[64,32,30,30]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,3,32,32]{3,2,1,0} %bitcast.2231, f32[32,3,3,3]{3,2,1,0} %bitcast.2238, f32[32]{0} %bitcast.2733), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivat

782/782 - 7s - 9ms/step - accuracy: 0.3716 - loss: 1.7167
Epoch 2/10
782/782 - 2s - 3ms/step - accuracy: 0.5054 - loss: 1.3837
Epoch 3/10
782/782 - 2s - 3ms/step - accuracy: 0.5529 - loss: 1.2593
Epoch 4/10
782/782 - 2s - 3ms/step - accuracy: 0.5886 - loss: 1.1687
Epoch 5/10
782/782 - 2s - 3ms/step - accuracy: 0.6187 - loss: 1.0887
Epoch 6/10
782/782 - 2s - 3ms/step - accuracy: 0.6402 - loss: 1.0328
Epoch 7/10
782/782 - 2s - 3ms/step - accuracy: 0.6604 - loss: 0.9778
Epoch 8/10
782/782 - 2s - 3ms/step - accuracy: 0.6769 - loss: 0.9291
Epoch 9/10
782/782 - 2s - 3ms/step - accuracy: 0.6951 - loss: 0.8839
Epoch 10/10
782/782 - 2s - 3ms/step - accuracy: 0.7067 - loss: 0.8460


2025-05-27 12:53:24.928202: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.9 = (f32[64,32,30,30]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,3,32,32]{3,2,1,0} %bitcast.526, f32[32,3,3,3]{3,2,1,0} %bitcast.533, f32[32]{0} %bitcast.535), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1/conv2d_1/convolution" source_file="/project/nanocourse/SWE_OOP/shared/CondaEnvs/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-05-27 12:53:24.937435: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted pote

157/157 - 2s - 10ms/step - accuracy: 0.6720 - loss: 0.9546


[0.9545750021934509, 0.671999990940094]

In [7]:
def my_model():
    inputs = keras.Input(shape=(32, 32, 3))
    x = layers.Conv2D(32, 3)(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(10)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


modelFunc = my_model()
modelFunc.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)


In [8]:
print(modelFunc.summary())


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 30, 30, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 13, 13, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 13, 13, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 4, 4, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       131,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,930 (882.54 KB)

 Trainable params: 225,482 (880.79 KB)

 Non-trainable params: 448 (1.75 KB)

None


In [9]:
modelFunc.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
modelFunc.evaluate(x_test, y_test, batch_size=64, verbose=2)


Epoch 1/10
782/782 - 9s - 11ms/step - accuracy: 0.5309 - loss: 1.3114
Epoch 2/10
782/782 - 3s - 3ms/step - accuracy: 0.6740 - loss: 0.9375
Epoch 3/10
782/782 - 3s - 3ms/step - accuracy: 0.7246 - loss: 0.7928
Epoch 4/10
782/782 - 3s - 3ms/step - accuracy: 0.7598 - loss: 0.6889
Epoch 5/10
782/782 - 3s - 3ms/step - accuracy: 0.7885 - loss: 0.6095
Epoch 6/10
782/782 - 3s - 3ms/step - accuracy: 0.8136 - loss: 0.5405
Epoch 7/10
782/782 - 3s - 3ms/step - accuracy: 0.8364 - loss: 0.4745
Epoch 8/10
782/782 - 3s - 3ms/step - accuracy: 0.8544 - loss: 0.4226
Epoch 9/10
782/782 - 3s - 3ms/step - accuracy: 0.8760 - loss: 0.3691
Epoch 10/10
782/782 - 3s - 3ms/step - accuracy: 0.8928 - loss: 0.3221
157/157 - 2s - 10ms/step - accuracy: 0.6879 - loss: 1.0656


[1.0655699968338013, 0.6879000067710876]

In [10]:
# To Do:
# 1. construct a new class (type) called MyModelSubClass which is a keras.Model, instantiate it,
# and then fit the model to the data.
#
# 2. Now explore: See what kind of accuracy you can get by training longer,
# increasing model size, changing the kernel sizes.


In [11]:
class MyModelSubClass(keras.Model):
    def __init__(self, name="MyModelSubclass", **kwargs):
        # call super (base) class's init
        super().__init__()

        # define layers to be used in forward pass\
        self.conv2d_1 = layers.Conv2D(32, 3)
        self.batch_norm_1 = layers.BatchNormalization()
        self.relu = keras.activations.relu
        self.conv2d_2 = layers.Conv2D(64, 3)
        self.batch_norm_2 = layers.BatchNormalization()
        self.maxpooling2d = layers.MaxPooling2D()
        self.conv2d_3 = layers.Conv2D(128, 3)
        self.batch_norm_3 = layers.BatchNormalization()
        self.flatten = layers.Flatten()
        self.dense = layers.Dense(64, activation="relu")
        self.outputs = layers.Dense(10)

    def call(self, inputs, training=None):
        # return the prediction
        x = self.conv2d_1(inputs)
        x = self.batch_norm_1(x, training=training)
        x = self.relu(x)
        x = self.maxpooling2d(x)
        x = self.conv2d_2(x)
        x = self.batch_norm_2(x)
        x = self.relu(x)
        x = self.maxpooling2d(x)
        x = self.conv2d_3(x)
        x = self.batch_norm_3(x)
        x = self.relu(x)
        x = self.flatten(x)
        x = self.dense(x)
        outputs = self.outputs(x)

        return outputs

    def model(self):
        x = keras.Input(shape=(32, 32, 3))
        return keras.Model(inputs=[x], outputs=self.call(x))


In [12]:
# Instantiate the model
mod = MyModelSubClass()


In [13]:
# Print a summary of the model
mod.model().summary()
# Compile the model
mod.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 30, 30,    │        896 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 30, 30,    │        128 │ conv2d_6[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 30, 30,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 6, 6, 64)  │          0 │ re_lu_3[0][0],    │
│ (MaxPooling2D)      │                   │            │ re_lu_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 13, 13,    │     18,496 │ max_pooling2d_4[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 13, 13,    │        256 │ conv2d_7[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_4 (ReLU)      │ (None, 13, 13,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 4, 4, 128) │     73,856 │ max_pooling2d_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 4, 4, 128) │        512 │ conv2d_8[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_5 (ReLU)      │ (None, 4, 4, 128) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 2048)      │          0 │ re_lu_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │    131,136 │ flatten_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 10)        │        650 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 225,930 (882.54 KB)

 Trainable params: 225,482 (880.79 KB)

 Non-trainable params: 448 (1.75 KB)

In [14]:
# Fit the model
model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)


Epoch 1/10
782/782 - 2s - 3ms/step - accuracy: 0.7217 - loss: 0.8075
Epoch 2/10
782/782 - 2s - 3ms/step - accuracy: 0.7338 - loss: 0.7713
Epoch 3/10
782/782 - 2s - 3ms/step - accuracy: 0.7425 - loss: 0.7448
Epoch 4/10
782/782 - 2s - 3ms/step - accuracy: 0.7527 - loss: 0.7131
Epoch 5/10
782/782 - 2s - 3ms/step - accuracy: 0.7643 - loss: 0.6803
Epoch 6/10
782/782 - 2s - 3ms/step - accuracy: 0.7748 - loss: 0.6536
Epoch 7/10
782/782 - 2s - 3ms/step - accuracy: 0.7856 - loss: 0.6249
Epoch 8/10
782/782 - 2s - 3ms/step - accuracy: 0.7915 - loss: 0.6041
Epoch 9/10
782/782 - 2s - 3ms/step - accuracy: 0.7998 - loss: 0.5797
Epoch 10/10
782/782 - 2s - 3ms/step - accuracy: 0.8070 - loss: 0.5562


In [15]:
# Check your model's performance
model.evaluate(x_test, y_test, batch_size=64, verbose=2)


157/157 - 0s - 2ms/step - accuracy: 0.7278 - loss: 0.8175


[0.8175175189971924, 0.7278000116348267]

In [16]:
### Rinse and repeat; this time add more layers.


In [17]:
class MyModelSubClass_Layers(keras.Model):
    def __init__(self, name="MyModelSubClass_Layers", **kwargs):
        # call super (base) class's init
        super().__init__()

        # define layers to be used in forward pass
        # define layers to be used in forward pass\
        self.conv2d_1 = layers.Conv2D(32, 3)
        self.batch_norm_1 = layers.BatchNormalization()
        self.relu = keras.activations.relu
        self.conv2d_2 = layers.Conv2D(64, 3)
        self.batch_norm_2 = layers.BatchNormalization()
        self.maxpooling2d = layers.MaxPooling2D()
        self.conv2d_3 = layers.Conv2D(128, 3)
        self.batch_norm_3 = layers.BatchNormalization()
        self.flatten = layers.Flatten()
        self.dense = layers.Dense(64, activation="relu")
        self.dense2 = layers.Dense(32, activation="relu")
        self.outputs = layers.Dense(10)

    def call(self, inputs, training=None):
        x = self.conv2d_1(inputs)
        x = self.batch_norm_1(x, training=training)
        x = self.relu(x)
        x = self.maxpooling2d(x)
        x = self.conv2d_2(x)
        x = self.batch_norm_2(x)
        x = self.relu(x)
        x = self.maxpooling2d(x)
        x = self.conv2d_3(x)
        x = self.batch_norm_3(x)
        x = self.relu(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.dense2(x)
        outputs = self.outputs(x)
        # return the prediction
        return outputs

    def model(self):
        x = keras.Input(shape=(32, 32, 3))
        return keras.Model(inputs=[x], outputs=self.call(x))


In [18]:
# Instantiate the model
mod = MyModelSubClass_Layers()


In [19]:
# Print a summary of the model
mod.model().summary()

# Compile the model
mod.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 30, 30,    │        896 │ input_layer_3[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 30, 30,    │        128 │ conv2d_9[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_6 (ReLU)      │ (None, 30, 30,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 6, 6, 64)  │          0 │ re_lu_6[0][0],    │
│ (MaxPooling2D)      │                   │            │ re_lu_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 13, 13,    │     18,496 │ max_pooling2d_5[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 13, 13,    │        256 │ conv2d_10[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_7 (ReLU)      │ (None, 13, 13,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 4, 4, 128) │     73,856 │ max_pooling2d_5[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 4, 4, 128) │        512 │ conv2d_11[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_8 (ReLU)      │ (None, 4, 4, 128) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 2048)      │          0 │ re_lu_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 64)        │    131,136 │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 32)        │      2,080 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 10)        │        330 │ dense_7[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 227,690 (889.41 KB)

 Trainable params: 227,242 (887.66 KB)

 Non-trainable params: 448 (1.75 KB)

In [20]:
# Fit the model
mod.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)


Epoch 1/10
782/782 - 9s - 12ms/step - accuracy: 0.5091 - loss: 1.3865
Epoch 2/10
782/782 - 3s - 3ms/step - accuracy: 0.6566 - loss: 0.9803
Epoch 3/10
782/782 - 3s - 3ms/step - accuracy: 0.7181 - loss: 0.8138
Epoch 4/10
782/782 - 3s - 3ms/step - accuracy: 0.7508 - loss: 0.7156
Epoch 5/10
782/782 - 3s - 3ms/step - accuracy: 0.7792 - loss: 0.6330
Epoch 6/10
782/782 - 3s - 3ms/step - accuracy: 0.8047 - loss: 0.5642
Epoch 7/10
782/782 - 3s - 3ms/step - accuracy: 0.8259 - loss: 0.5035
Epoch 8/10
782/782 - 3s - 3ms/step - accuracy: 0.8456 - loss: 0.4522
Epoch 9/10
782/782 - 3s - 3ms/step - accuracy: 0.8629 - loss: 0.4008
Epoch 10/10
782/782 - 3s - 3ms/step - accuracy: 0.8791 - loss: 0.3546


In [21]:
# Check your model's performance. How does the performance compare with the smaller model?
mod.evaluate(x_test, y_test, batch_size=64, verbose=2)


157/157 - 2s - 11ms/step - accuracy: 0.7128 - loss: 0.9462


[0.9462383985519409, 0.7128000259399414]

In [22]:
### Rinse and repeat; this time change the kernel size.


In [23]:
class MyModelSubClass_Kernel(keras.Model):
    def __init__(self, name="MyModelSubClass_Kernel", **kwargs):
        # call super (base) class's init
        super().__init__()

        # define layers to be used in forward pass
        self.conv2d_1 = layers.Conv2D(32, 5)
        self.batch_norm_1 = layers.BatchNormalization()
        self.relu = keras.activations.relu
        self.conv2d_2 = layers.Conv2D(64, 5)
        self.batch_norm_2 = layers.BatchNormalization()
        self.maxpooling2d = layers.MaxPooling2D()
        self.conv2d_3 = layers.Conv2D(128, 5)
        self.batch_norm_3 = layers.BatchNormalization()
        self.flatten = layers.Flatten()
        self.dense = layers.Dense(64, activation="relu")
        self.outputs = layers.Dense(10)

    def call(self, inputs, training=None):
        x = self.conv2d_1(inputs)
        x = self.batch_norm_1(x, training=training)
        x = self.relu(x)
        x = self.maxpooling2d(x)
        x = self.conv2d_2(x)
        x = self.batch_norm_2(x)
        x = self.relu(x)
        x = self.maxpooling2d(x)
        x = self.conv2d_3(x)
        x = self.batch_norm_3(x)
        x = self.relu(x)
        x = self.flatten(x)
        x = self.dense(x)
        outputs = self.outputs(x)
        # return the prediction
        return outputs

    def model(self):
        x = keras.Input(shape=(32, 32, 3))
        return keras.Model(inputs=[x], outputs=self.call(x))


In [24]:
# Instantiate the model
mod = MyModelSubClass_Kernel()


In [25]:
# Print a summary of the model
mod.model().summary()
# Compile the model
mod.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 28, 28,    │      2,432 │ input_layer_4[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 28, 28,    │        128 │ conv2d_12[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_9 (ReLU)      │ (None, 28, 28,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 5, 5, 64)  │          0 │ re_lu_9[0][0],    │
│ (MaxPooling2D)      │                   │            │ re_lu_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 10, 10,    │     51,264 │ max_pooling2d_6[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 10, 10,    │        256 │ conv2d_13[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_10 (ReLU)     │ (None, 10, 10,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 1, 1, 128) │    204,928 │ max_pooling2d_6[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1, 1, 128) │        512 │ conv2d_14[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_11 (ReLU)     │ (None, 1, 1, 128) │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_4 (Flatten) │ (None, 128)       │          0 │ re_lu_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 64)        │      8,256 │ flatten_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 10)        │        650 │ dense_9[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 268,426 (1.02 MB)

 Trainable params: 267,978 (1.02 MB)

 Non-trainable params: 448 (1.75 KB)

In [26]:
# Fit the model
mod.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)


Epoch 1/10


2025-05-27 12:55:08.156406: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.6 = (f32[64,32,28,28]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,3,32,32]{3,2,1,0} %bitcast.3651, f32[32,3,5,5]{3,2,1,0} %bitcast.3658, f32[32]{0} %bitcast.4572), window={size=5x5}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="my_model_sub_class__kernel_1/conv2d_12_1/convolution" source_file="/project/nanocourse/SWE_OOP/shared/CondaEnvs/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-05-27 12:55:08.160813: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picke

782/782 - 10s - 12ms/step - accuracy: 0.5319 - loss: 1.3162
Epoch 2/10
782/782 - 3s - 3ms/step - accuracy: 0.6670 - loss: 0.9489
Epoch 3/10
782/782 - 3s - 3ms/step - accuracy: 0.7199 - loss: 0.7960
Epoch 4/10
782/782 - 3s - 3ms/step - accuracy: 0.7603 - loss: 0.6914
Epoch 5/10
782/782 - 3s - 3ms/step - accuracy: 0.7892 - loss: 0.6018
Epoch 6/10
782/782 - 3s - 3ms/step - accuracy: 0.8160 - loss: 0.5312
Epoch 7/10
782/782 - 3s - 3ms/step - accuracy: 0.8361 - loss: 0.4698
Epoch 8/10
782/782 - 3s - 3ms/step - accuracy: 0.8579 - loss: 0.4118
Epoch 9/10
782/782 - 3s - 3ms/step - accuracy: 0.8763 - loss: 0.3622
Epoch 10/10
782/782 - 3s - 3ms/step - accuracy: 0.8905 - loss: 0.3183


In [28]:
# Check your model's performance. How does the performance compare with the other two models?
mod.evaluate(x_test, y_test, batch_size=64, verbose=2)


157/157 - 0s - 2ms/step - accuracy: 0.6541 - loss: 1.2311


[1.2311053276062012, 0.6541000008583069]